# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance
from sklearn.metrics import r2_score
from sklearn.datasets import make_spd_matrix
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [2]:
def data_read(file):
    """
    input:        file [object] - путь к файлу
    output:       data [DataFrame] - таблица данных
    description:  функция загружает таблицу функцией read_csv, выводит первые 10 строк на экран,
                  выводит информацию о данных методом info(), считает количество дубликатов
    """
    # читаем данные из .csv файла с помощью метода read_csv()
    data = pd.read_csv(file)
    display(data.head(10))
    print(data.info())
    print('Количество дубликатов:', sum(data.duplicated()))
    return data

In [3]:
data = data_read('/datasets/insurance.csv')


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB
None
Количество дубликатов: 153


- Данные разделены на 4 признака (пол, возраст, зарплата и члены семьи) и целевой признак (страховые выплаты)
- Пропуски отсутствуют
- Все признаки - количественные 
- В данных присутствуют дубликаты, их стоит удалить 
- Так же стоит переименовать столбцы для соблюдения аккуратности кода


In [4]:
#удалим дубликаты
data.drop_duplicates(inplace=True)
data.reset_index(inplace=True, drop=True)

#переименуем столбцы
data.rename(columns={'Пол': 'sex', 'Возраст': 'age', 'Зарплата': 'salary', 'Члены семьи': 'family_members', 
                     'Страховые выплаты': 'insurance_benefits'}, inplace=True)
data.sample()

,sex,age,salary,family_members,insurance_benefits
2711,0,38.0,34100.0,1,0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4847 entries, 0 to 4846
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   sex                 4847 non-null   int64  
 1   age                 4847 non-null   float64
 2   salary              4847 non-null   float64
 3   family_members      4847 non-null   int64  
 4   insurance_benefits  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 189.5 KB


## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество модели не изменится


**Обоснование:** 

Новая матрица признаков:

$$
X' = XP
$$

Новый вектор весов $w'$ можем расчитать следующим образом:

$$
w' = ((XP)^T XP)^{-1} (XP)^T y == (P^T X^T XP)^{-1}P^T X^T y == P^{-1}(X^T X)^{-1} (P^T)^{-1} P^T X^T y == P^{-1} w
$$

Внутри формулы можно выделить формулу расчета $w$

$$
(P^T)^{-1} P^T = E =1
$$

таким образом:

$$
a' = X'w' = XPP^{-1} w = XEw = Xw = a
$$


Можно отметить, что, если любую матрицу Р умножить на единичную (или наоборот), получится эта же матрица Р. Т.е., матрицы будут одинаковыми, а качество линейной регрессии не изменится.



**Вывод:**

Предсказания модели не изменилист, а значит и качество остается тем же.

## Алгоритм преобразования

**Алгоритм**

1) методом make_spd_matrix создаем случайную матрицу A

2) методом np.linalg.inv() проверяем матрицу А на обратимость и получаем $A^{-1}$. Если матрица А провалила тест на обратимость, необходимо создать новую случайную матрицу.

Поскольку нам надо закодировать данные клиентов, матрица А будет ключом кодирования.

3) умножаем матрицу признаков X на полученную матрицу А и получаем матрицу B с закодированными данными о клиентах:

$$
B = X@A
$$

После этого можно переходить к обучению модели линейной регрессии.

**Обоснование**

Такой алгоритм позволит защитить персональную информацию клиентов и при этом не должен повлиять на работу модели (ранее уже отмечалось, что качество модели не изменится, если матрицу признаков умножать на обратимую модель).

## Проверка алгоритма

### Преобразование данных

In [6]:
features = data.drop('insurance_benefits', axis=1)
target = data['insurance_benefits']

print('Признаки', features.shape)
print('Целевой признак', target.shape)

Признаки (4847, 4)
Целевой признак (4847,)


In [7]:
#создаем матрицу признаков

features_matrix = features.values
features_matrix

#создаем случайную матрицу.

random_matrix = make_spd_matrix(n_dim=4, random_state=123)
random_matrix

#проверяем на обратимость

np.linalg.inv(random_matrix)

array([[ 2.97675307,  0.09601407, -0.43363796,  1.63149048],
       [ 0.09601407,  1.02415391, -1.2595126 ,  0.57122764],
       [-0.43363796, -1.2595126 ,  2.39108163, -0.46226306],
       [ 1.63149048,  0.57122764, -0.46226306,  4.18355526]])

Матрица обратима, так как не возникло ошибки 

In [8]:
#умножим матрицу признаков features_matrix на матрицу random_matrix. Результат сохраним в переменной encoded_matrix.

encoded_matrix = features_matrix @ random_matrix
encoded_matrix

type(encoded_matrix)

numpy.ndarray

###  Обучение модели

*Исходные данные*

Обучим модель линейной регрессии на исходных данных.

In [12]:
#Разделим данные на обучающую и валидационные выборки
features_train, features_valid, target_train, target_valid = train_test_split(features, 
                                                                              target, 
                                                                              train_size=0.75, 
                                                                              test_size=0.25, 
                                                                              random_state=123)

#Выведем на экран размер выборок.

print('features_train:', features_train.shape) 
print('target_train:', target_train.shape)

print('features_valid:', features_valid.shape)
print('target_valid:', target_valid.shape)

features_train: (3635, 4)
target_train: (3635,)
features_valid: (1212, 4)
target_valid: (1212,)


In [17]:
#Обучим модель и посчитаем метрику R2.

model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
r2 = r2_score(target_valid, predictions)

print(r2)

0.40978958053663006


Метрика не больше 1 и не меньше 0, значит модель обучена правильно. 

*Преобразованные данные*

Проверим метрика R2 на преобразованных данных.

In [18]:
#Разделим данные на обучающую и валидационные выборки
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(encoded_matrix, 
                                                                                      target, 
                                                                                      train_size=0.75, 
                                                                                      test_size=0.25, 
                                                                                      random_state=123)

#Выведем на экран размер выборок.
print('features_train_1:', features_train_1.shape) 
print('target_train_1:', target_train_1.shape)
print('features_valid_1:', features_valid_1.shape)
print('target_valid_1:', target_valid_1.shape)

features_train_1: (3635, 4)
target_train_1: (3635,)
features_valid_1: (1212, 4)
target_valid_1: (1212,)


In [19]:
#Обучим модель и посчитаем метрику R2.

model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_1 = model_1.predict(features_valid_1)
r2_1 = r2_score(target_valid_1, predictions_1)

print(r2)

0.40978958053663006


***Вывод***

Метрики идентичны, а это значит, что качество модели линейной регрессии после преобразования данных не изменилось. Но стотит отметить, что сам показатель метрики R2 достаточно низкий, значит, качество модели можно значительно улучшить, например, меняя гиперпараметры.